In [85]:
# Import Packages
import pandas
import pickle
import nltk
from tqdm.notebook import tqdm
import numpy as np
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import swifter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [87]:
# Import the Raw DataFrame
dataFile = open("df_all_sentiment_twitter-roberta-base-sentiment.pkl", 'rb')
df = pickle.load(dataFile)
df['Original_Index'] = range(len(df))
df

,sentence,ACN,positive_sentiment_score,neutral_sentiment_score,negative_sentiment_score,Original_Index
0,the gentleman i spoke with at lindbergh twr ov...,75085,0.814141,NaN,NaN,0
1,"""""approx 5 mi nw of the mission bay vor"", whil...",75085,NaN,0.033765,NaN,1
2,after approx 5 attempts i was able to reach a ...,75085,NaN,0.256700,NaN,2
3,w/o hesitation i quickly changed freqs to find...,75085,NaN,0.139586,NaN,3
4,as i was trying to contact the third san apch ...,75085,NaN,0.046918,NaN,4
...,...,...,...,...,...,...
3086614,"tucson and santa barbara have the same problem.""""",1925323,NaN,NaN,0.021431,3086614
3086615,"""""multiple aircraft have landed over multiple ...",1925480,NaN,0.054096,NaN,3086615
3086616,it is listed on the sectional as the first fre...,1925480,NaN,0.039939,NaN,3086616
3086617,it really shouldn't be on the sectional at all,1925480,NaN,NaN,0.005559,3086617


In [3]:
# Get List of Stop Words From NLTK Package
stopWords = set(stopwords.words('english'))

In [10]:
# replacing = {'w/o': 'without', 'twr': 'tower', 'freqs': 'frequencies', 'ctlr' : 'controller',
             # 'alt': 'altitude', 'approx': 'approximately','nw': 'north west', 'twrs': 'towers', 
             # 'clrnc': 'clearance', 'ctlrs': 'controllers', 'freq': 'frequency', 'plts': 'pilots', 
             # 'lcl': 'local', 'rpt': 'report',  'acft': 'aircraft', 'dscnt': 'descent', 'spd': 'speed',
             # 'kts': 'knots', 'deg': 'degree', 'nav': 'navigation', 'xing': 'crossing', 'kt': 'knot', 
             # 'lndg': 'landing', 'tca':'traffic control area', 'mi': 'mile', 'turb': 'turbulence', 'flt': 'flight', 
             # 'apch': 'approach', 'ctl': 'control', 'pax': 'passenger', 'ils': 'instrument landing system',
             # 'arpt': 'airport', 'coplt': 'copilot', 'atc': 'air traffic control', 'ne': 'north east',
             # 'sw': 'south west', 'rwy': 'runway', 'rwys': 'runways', 'ident': 'identify', 'prob': 'problem', 
             # 'ctled': 'controlled', 'info': 'information', 'tkof': 'takeoff', 'clrd': 'cleared', 'r': 'our', 
             # 'fms': 'flight management system', 'maint': 'maintenance', 'ca': 'california', 'a/c': 'aircraft', 
             # 'tfc': 'traffic', 'hdg': 'heading', 'apching': 'approaching', 'prox': 'proximity', 'des': 'descent', 
             # 'fpm': 'feet per minute', 'f/o': 'first officer', 'ft': 'feet', 'chklists': 'checklists', 
             # 'acn': 'aircraft classification number'}
lemmatizer = WordNetLemmatizer()
# split sentence into tokens
def TOKENIZE(row):
    fixed = []
    sentence = row.sentence
    for word in sentence.split():
        word = word.replace('"', '')
        word = word.replace(',', '')
        word = word.replace("\\", '')
        word = word.replace('(', '')
        word = word.replace(')', '')
        word = word.replace(':', '')
        word = word.replace('!', '')
        word = word.replace('?', '')
        word = word.replace('.', '')
        word = word.replace("'", '')
        word = word.replace('-', '')
        word = word.replace('[', '')
        word = word.replace(']', '')
        # if word.lower() in replacing: word = replacing[word]
        if(any(char.isdigit() for char in word)): continue
        if wordnet.synsets(word):lemmatizer.lemmatize(word)
        fixed.append(word)
        
  
    sentence = ' '.join(fixed)
    return sentence.split()

In [11]:
# Remove Stop Words from Tokenized Sentence
def REMOVE_STOP(row):
    filtered_tokens = []
    tokens = TOKENIZE(row)
    for token in tokens:
        if(token not in stopWords):
            filtered_tokens.append(token)
    return filtered_tokens

In [12]:
# Get Parts of Speech For Primitive Sentence Verification
def GET_POS(filtered_tokens: list):
    POS_RAW = pos_tag(filtered_tokens)
    POS = []
    for i in range(len(POS_RAW)):
        POS.append(POS_RAW[i][1])
    return POS

In [13]:
# Validate Processed Sentence to see if primitive sentence
def VALID_PRIMITIVE_SENTENCE(POS):
    if ('NN' in POS or 'NNS' in POS or 'NNP' in POS or 'NNPS' in POS) and ('VB' in POS or 'VBD' in POS or 'VBG' in POS or 'VBN' in POS or 'VBP' in POS or 'VBZ' in POS):
        return True
    return False

In [14]:
def PREPROCESS_SENTENCES(row):
    if not np.isnan(row['negative_sentiment_score']):
        return 'DROP'
    
    filtered_tokens = REMOVE_STOP(row)
    
    POS = GET_POS(filtered_tokens)
    if not VALID_PRIMITIVE_SENTENCE(POS):
        return 'DROP'

    
    return ' '.join(filtered_tokens)

In [88]:
df['sentence'] = df.swifter.apply(PREPROCESS_SENTENCES, axis=1)

Pandas Apply:   0%|          | 0/3086619 [00:00<?, ?it/s]

In [91]:
drop_indexes = list(df[df['sentence'] == 'DROP'].index.values)
df = df.drop(drop_indexes)
df

,sentence,ACN,positive_sentiment_score,neutral_sentiment_score,negative_sentiment_score,Original_Index
0,gentleman spoke lindbergh twr phone polite see...,75085,0.814141,NaN,NaN,0
1,approx mi nw mission bay vor alt approx san pr...,75085,NaN,0.033765,NaN,1
3,w/o hesitation quickly changed freqs find rout...,75085,NaN,0.139586,NaN,3
4,trying contact third san apch ctlr noticed com...,75085,NaN,0.046918,NaN,4
5,several attempts contact third ctlr received f...,75085,NaN,0.041111,NaN,5
...,...,...,...,...,...,...
3086606,let us fly approach vectored commuter front us...,1925323,NaN,0.246761,NaN,3086606
3086607,could avoided provided information prior conta...,1925323,NaN,0.039105,NaN,3086607
3086609,digital atis need clearly tell crews approach ...,1925323,NaN,0.042575,NaN,3086609
3086615,multiple aircraft landed multiple years fbo fr...,1925480,NaN,0.054096,NaN,3086615


In [94]:
df.to_csv("processed_new.csv", index=False)

In [95]:
df2 = pandas.read_csv('processed_new.csv')
df2 

,sentence,ACN,positive_sentiment_score,neutral_sentiment_score,negative_sentiment_score,Original_Index
0,gentleman spoke lindbergh twr phone polite see...,75085,0.814141,NaN,NaN,0
1,approx mi nw mission bay vor alt approx san pr...,75085,NaN,0.033765,NaN,1
2,w/o hesitation quickly changed freqs find rout...,75085,NaN,0.139586,NaN,3
3,trying contact third san apch ctlr noticed com...,75085,NaN,0.046918,NaN,4
4,several attempts contact third ctlr received f...,75085,NaN,0.041111,NaN,5
...,...,...,...,...,...,...
2328174,let us fly approach vectored commuter front us...,1925323,NaN,0.246761,NaN,3086606
2328175,could avoided provided information prior conta...,1925323,NaN,0.039105,NaN,3086607
2328176,digital atis need clearly tell crews approach ...,1925323,NaN,0.042575,NaN,3086609
2328177,multiple aircraft landed multiple years fbo fr...,1925480,NaN,0.054096,NaN,3086615


In [96]:
df2[df2['sentence']=='DROP']

,sentence,ACN,positive_sentiment_score,neutral_sentiment_score,negative_sentiment_score,Original_Index
